In [1]:
# make sure we are working in module directory
repo_root = !git rev-parse --show-toplevel
module_path = repo_root[0] + "/backend/heatflask"
%cd $module_path

/home/efrem/dev/heatflask/backend/heatflask


In [2]:
# %load Users.py
"""
***  For Jupyter notebook ***
Paste one of these Jupyter magic directives to the top of a cell
 and run it, to do these things:
    %%cython --annotate       # Compile and run the cell
    %load Users.py            # Load Users.py file into this (empty) cell
    %%writefile Users.py      # Write the contents of this cell to Users.py
"""

from logging import getLogger
import datetime

import DataAPIs
import Utility

log = getLogger(__name__)
log.propagate = True

COLLECTION_NAME = "users"

# Drop a user after a year of inactivity
MONGO_TTL = 365 * 24 * 3600

ADMIN = [15972102]


class Box:
    collection = None


myBox = Box()


async def get_collection():
    if myBox.collection is None:
        myBox.collection = await DataAPIs.init_collection(
            COLLECTION_NAME, ttl=MONGO_TTL
        )
    return myBox.collection


def mongo_doc(
    # From Strava Athlete record
    id=None,
    username=None,
    firstname=None,
    lastname=None,
    profile_medium=None,
    profile=None,
    measurement_preference=None,
    city=None,
    state=None,
    country=None,
    email=None,
    # my additions
    _id=None,
    ts=None,
    auth=None,
    access_count=None,
    private=None,
    **extras
):
    if not (id or _id):
        log.error("cannot create user with no id")
        return

    return Utility.cleandict(
        {
            "_id": int(_id or id),
            "username": username,
            "firstname": firstname,
            "lastname": lastname,
            "profile": profile_medium or profile,
            "units": measurement_preference,
            "city": city,
            "state": state,
            "country": country,
            "email": email,
            #
            "ts": ts,
            "access_count": access_count,
            "auth": auth,
            "private": private,
        }
    )


async def add_or_update(update_ts=False, inc_access_count=False, **userdict):
    users = await get_collection()
    doc = mongo_doc(**userdict)
    if not doc:
        log.exception("error adding/updating user: %s", doc)
        return

    user_id = doc.pop("_id")

    if update_ts:
        doc["ts"] = datetime.datetime.utcnow()

    updates = {"$set": doc} if doc else {}

    if inc_access_count:
        updates["$inc"] = {"access_count": 1}

    log.debug("calling mongodb update_one with updates %s", updates)

    # Creates a new user or updates an existing user (with the same id)
    try:
        return await users.update_one({"_id": user_id}, updates, upsert=True)
    except Exception:
        log.exception("error adding/updating user: %s", doc)


async def get(user_id):
    users = await get_collection()
    uid = int(user_id)
    query = {"_id": uid}
    try:
        doc = await users.find_one(query)
    except Exception:
        log.exception("Failed mongodb query: %s", query)
        doc = None
    return doc


# Returns an async iterator
async def get_all():
    users = await get_collection()
    return users.find()


async def delete(user_id):
    users = await get_collection()
    uid = int(user_id)
    try:
        return await users.delete_one({"_id": uid})

    except Exception:
        log.exception("error deleting user %d", uid)


def stats():
    return DataAPIs.stats(COLLECTION_NAME)


def drop():
    return DataAPIs.drop(COLLECTION_NAME)


In [3]:
import logging
logging.basicConfig(level="DEBUG")

await DataAPIs.connect()

new_user_dict = {"username": "guy"}
await add_or_update(**new_user_dict)  # should give an error

new_user_dict = {**new_user_dict, "id": 222}
await add_or_update(**new_user_dict)

DEBUG:DataAPIs:initialized MongoDB and Redis
ERROR:__main__:cannot create user with no id
ERROR:__main__:error adding/updating user: None
NoneType: None
DEBUG:__main__:calling mongodb update_one with updates {'$set': {'username': 'guy'}}


In [4]:
await get(222)

{'_id': 222, 'username': 'guy'}

In [5]:
await add_or_update(_id=222, private=False)

await add_or_update(id=222, inc_access_count=True)

result = await add_or_update(id=222, update_ts=True)
user_info = await get(222)
result.raw_result, user_info

DEBUG:__main__:calling mongodb update_one with updates {'$set': {'private': False}}
DEBUG:__main__:calling mongodb update_one with updates {'$inc': {'access_count': 1}}
DEBUG:__main__:calling mongodb update_one with updates {'$set': {'ts': datetime.datetime(2022, 2, 15, 18, 56, 6, 957613)}}


({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True},
 {'_id': 222,
  'username': 'guy',
  'private': False,
  'access_count': 1,
  'ts': datetime.datetime(2022, 2, 15, 18, 56, 6, 957000)})

In [6]:
result = await delete(222)
result.raw_result

{'n': 1, 'ok': 1.0}

In [8]:
# import real user data from Strava
import Strava
print("Paste this URL into your browser and retrieve the code:\n", Strava.auth_url())

Paste this URL into your browser and retrieve the code:
 https://www.strava.com/oauth/authorize?client_id=12700&response_type=code&approval_prompt=force&scope=read,activity:read,activity:read_all&redirect_uri=http:%2F%2Flocalhost%2Fexchange_token


In [9]:
# exchange code for token
CODE = "7eb4e6a3f74ef50947f39bda8660ba3400d2723b"
C = Strava.AsyncClient("admin")
new_info = await C.update_access_token(code=CODE)
user_dict = new_info.pop("athlete")
new_info, user_dict

DEBUG:Strava:refreshing access token from code
INFO:Strava:admin token refresh took 507


({'expires_at': 1644955224,
  'refresh_token': '05867993a2d0c5b60c51653636a9c295348551f3',
  'access_token': '5dabace655c26d1372321c53f5dad173ec2bd674'},
 {'id': 15972102,
  'username': 'bfef',
  'resource_state': 2,
  'firstname': '👣',
  'lastname': 'Efrem',
  'bio': '* runs barefoot 👣\n\nhttps://www.heatflask.com',
  'city': 'Oakland',
  'state': 'California',
  'country': 'United States',
  'sex': 'M',
  'premium': True,
  'summit': True,
  'created_at': '2016-06-25T03:48:55Z',
  'updated_at': '2021-09-27T06:08:53Z',
  'badge_type_id': 1,
  'weight': 81.22,
  'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg',
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/large.jpg',
  'friend': None,
  'follower': None})

In [10]:
mongo_doc(**user_dict)

{'_id': 15972102,
 'username': 'bfef',
 'firstname': '👣',
 'lastname': 'Efrem',
 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg',
 'city': 'Oakland',
 'state': 'California',
 'country': 'United States'}

In [11]:
await add_or_update(
    auth=new_info,
     update_ts=True, 
    inc_access_count=True,
    **user_dict, 
)

DEBUG:__main__:calling mongodb update_one with updates {'$set': {'username': 'bfef', 'firstname': '👣', 'lastname': 'Efrem', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg', 'city': 'Oakland', 'state': 'California', 'country': 'United States', 'auth': {'expires_at': 1644955224, 'refresh_token': '05867993a2d0c5b60c51653636a9c295348551f3', 'access_token': '5dabace655c26d1372321c53f5dad173ec2bd674'}, 'ts': datetime.datetime(2022, 2, 15, 18, 58, 32, 199696)}, '$inc': {'access_count': 1}}


In [12]:
# async_cursor = await get_all()
# userslist1 = [user async for user in async_cursor]  

async_cursor = await get_all()
userslist2 = await async_cursor.to_list(length=None)
userslist2

[{'_id': 15972102,
  'access_count': 2,
  'auth': {'expires_at': 1644955224,
   'refresh_token': '05867993a2d0c5b60c51653636a9c295348551f3',
   'access_token': '5dabace655c26d1372321c53f5dad173ec2bd674'},
  'city': 'Oakland',
  'country': 'United States',
  'firstname': '👣',
  'lastname': 'Efrem',
  'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/15972102/9131294/7/medium.jpg',
  'state': 'California',
  'ts': datetime.datetime(2022, 2, 15, 18, 58, 32, 199000),
  'username': 'bfef'}]

In [ ]:
await DataAPIs.disconnect()